<a href="https://colab.research.google.com/github/bio-null/screening-of-protein-design/blob/main/AFcluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AF-cluster (in Colab form!)

Last updated H Wayment-Steele, Jan 2023

### To cluster (subsample) a big MSA:

1. Upload the big MSA.

2. Run the "**Cluster MSA**" cell, changing `EX` to your desired keyword and the name of your input a3m. MSAs will be written to "subsampled_MSAs".

3. Uncomment the correct path (containing `/content/subsampled_MSAs` ) in the "**Run Models**" cell, and run it.

4. Output PDBs will be saved in `output`.

### If you have an existing set of subsampled MSAs to run:
1. Upload a directory of MSA files to run.

2. Change the `AFcluster/data..\*a3m` path in the "**Run Models**" cell and run it.

3. Output PDBs will be saved in `output`.

In [1]:
#@markdown ## Setup dependencies
%%bash

if [ ! -d params ]; then
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar | tar x -C params
fi

if [ ! -d AFcluster ];then
  git clone https://github.com/HWaymentSteele/AFcluster.git
fi

if [ ! -d alphafold ]; then
  git clone https://github.com/deepmind/alphafold.git
  ! pip -q install ml-collections dm-haiku biopython
fi

if [ ! -d output ]; then
  mkdir output
fi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 89.5 MB/s eta 0:00:00


Cloning into 'AFcluster'...
Updating files: 100% (13622/13622), done.
Cloning into 'alphafold'...


In [2]:
#@markdown ## Define functions to run AlphaFold

import sys
import os
import argparse
import hashlib
import jax
import jax.numpy as jnp
import numpy as np
import re
import subprocess
from glob import glob

sys.path.append('alphafold')

from alphafold.model import model, config, data
from alphafold.data import parsers, pipeline
from alphafold.common import protein

"""
Create an AlphaFold model runner
name -- The name of the model to get the parameters from. Options: model_[1-5]
"""

def make_model_runner(model_num=3, recycles=1, deterministic=True):
  model_name = 'model_%d_ptm' % model_num
  cfg = config.model_config(model_name)

  cfg.data.common.num_recycle = recycles
  cfg.model.num_recycle = recycles
  cfg.data.eval.num_ensemble = 1
  if deterministic:
    cfg.data.eval.masked_msa_replace_fraction = 0.0
    cfg.model.global_config.deterministic = True
  params = data.get_model_haiku_params(model_name, '.')

  return model.RunModel(cfg, params)

def make_processed_feature_dict(runner, a3m_file, name="test", seed=0):
  feature_dict = {}

  # assumes sequence is first entry in msa
  with open(a3m_file,'r') as msa_fil:
    sequence = msa_fil.read().splitlines()[1].strip()

  feature_dict.update(pipeline.make_sequence_features(sequence, name, len(sequence)))

  with open(a3m_file,'r') as msa_fil:
    msa = pipeline.parsers.parse_a3m(msa_fil.read())

  feature_dict.update(pipeline.make_msa_features([msa]))
  processed_feature_dict = runner.process_features(feature_dict, random_seed=seed)

  return processed_feature_dict

"""
Package AlphaFold's output into an easy-to-use dictionary
prediction_result - output from running AlphaFold on an input dictionary
processed_feature_dict -- The dictionary passed to AlphaFold as input. Returned by `make_processed_feature_dict`.
"""
def parse_results(prediction_result, processed_feature_dict):
  b_factors = prediction_result['plddt'][:,None] * prediction_result['structure_module']['final_atom_mask']
  dist_bins = jax.numpy.append(0,prediction_result["distogram"]["bin_edges"])
  dist_mtx = dist_bins[prediction_result["distogram"]["logits"].argmax(-1)]
  contact_mtx = jax.nn.softmax(prediction_result["distogram"]["logits"])[:,:,dist_bins < 8].sum(-1)

  out = {"unrelaxed_protein": protein.from_prediction(processed_feature_dict, prediction_result, b_factors=b_factors),
        "plddt": prediction_result['plddt'],
        "pLDDT": prediction_result['plddt'].mean(),
        "dists": dist_mtx,
        "adj": contact_mtx}

  out.update({"pae": prediction_result['predicted_aligned_error'],
              "pTMscore": prediction_result['ptm']})
  return out

def write_results(result, pdb_out_path):
  plddt = float(result['pLDDT'])
  ptm = float(result["pTMscore"])
  print('plddt: %.3f' % plddt)
  print('ptm  : %.3f' % ptm)

  pdb_lines = protein.to_pdb(result["unrelaxed_protein"])
  with open(pdb_out_path, 'w') as f:
    f.write(pdb_lines)

# **Cluster MSA**

In [3]:
%%bash

pip -q install polyleven

python AFcluster/scripts/ClusterMSA.py EX -i /content/1HAK_ab609.a3m -o subsampled_MSAs

EX
2581 seqs removed for containing more than 25% gaps, 14415 remaining
eps	n_clusters	n_not_clustered
3.00	1	3604
3.50	1	3604
4.00	2	3601
4.50	1	3604
5.00	1	3604
5.50	4	3591
6.00	6	3587
6.50	5	3590
7.00	3	3596
7.50	6	3579
8.00	5	3574
8.50	5	3572
9.00	7	3565
9.50	8	3555
10.00	19	3523
10.50	29	3467
11.00	76	3202
11.50	64	2603
12.00	50	1829
12.50	24	905
13.00	6	222
13.50	2	24
14.00	1	0
Selected eps=11.00
14415 total seqs
599 clusters, 9535 of 14415 not clustered (0.66)
avg identity to query of unclustered: 0.35
avg identity to query of clustered: 0.40
writing 10 size-10 uniformly sampled clusters
writing 10 size-100 uniformly sampled clusters
wrote clustering data to subsampled_MSAs/EX_clustering_assignments.tsv
wrote cluster metadata to subsampled_MSAs/EX_cluster_metadata.tsv
Saved this output to EX.log


/usr/local/lib/python3.12/dist-packages/Bio/SeqIO/FastaIO.py:202: BiopythonDeprecationWarning: Previously, the FASTA parser silently ignored comments at the beginning of the FASTA file (before the first sequence).

Nowadays, the FASTA file format is usually understood not to have any such comments, and most software packages do not allow them. Therefore, the use of comments at the beginning of a FASTA file is now deprecated in Biopython.

In a future Biopython release, this deprecation warning will be replaced by a ValueError. To avoid this, there are three options:

(1) Modify your FASTA file to remove such comments at the beginning of the file.

(2) Use SeqIO.parse with the 'fasta-pearson' format instead of 'fasta'. This format is consistent with the FASTA format defined by William Pearson's FASTA aligner software. This format allows for comments before the first sequence; lines starting with the ';' character anywhere in the file are also regarded as comment lines and are ignored.



In [9]:
n_recycles = 3
model_number = 3
seed=0
name='KaiB_TE'

runner = make_model_runner(model_num=model_number, recycles=n_recycles)

#subsampled_msas = glob('AFcluster/data_sep2022/00_KaiB/kaib_dbscan_msas/*a3m')
subsampled_msas = glob('subsampled_MSAs/EX_3[0-9][0-9].a3m')

for fil in subsampled_msas:
  print(fil)
  features = make_processed_feature_dict(runner, fil, seed=seed)
  result = parse_results(runner.predict(features, random_seed=seed), features)
  write_results(result, 'output/' + os.path.basename(fil).replace('.a3m','.pdb'))

subsampled_MSAs/EX_371.a3m
plddt: 52.535
ptm  : 0.474
subsampled_MSAs/EX_347.a3m
plddt: 58.500
ptm  : 0.591
subsampled_MSAs/EX_350.a3m
plddt: 69.338
ptm  : 0.700
subsampled_MSAs/EX_332.a3m
plddt: 70.151
ptm  : 0.692
subsampled_MSAs/EX_367.a3m
plddt: 62.330
ptm  : 0.625
subsampled_MSAs/EX_399.a3m
plddt: 81.360
ptm  : 0.814
subsampled_MSAs/EX_318.a3m
plddt: 71.783
ptm  : 0.721
subsampled_MSAs/EX_397.a3m
plddt: 72.034
ptm  : 0.716
subsampled_MSAs/EX_370.a3m
plddt: 60.513
ptm  : 0.598
subsampled_MSAs/EX_353.a3m
plddt: 82.165
ptm  : 0.814
subsampled_MSAs/EX_382.a3m
plddt: 78.431
ptm  : 0.790
subsampled_MSAs/EX_360.a3m
plddt: 79.082
ptm  : 0.779
subsampled_MSAs/EX_363.a3m
plddt: 92.211
ptm  : 0.878
subsampled_MSAs/EX_329.a3m
plddt: 75.373
ptm  : 0.749
subsampled_MSAs/EX_310.a3m
plddt: 59.255
ptm  : 0.591
subsampled_MSAs/EX_324.a3m
plddt: 69.561
ptm  : 0.680
subsampled_MSAs/EX_313.a3m
plddt: 70.779
ptm  : 0.709
subsampled_MSAs/EX_381.a3m
plddt: 69.744
ptm  : 0.719
subsampled_MSAs/EX_300.a3m
p

In [12]:
!zip -r result3.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/EX_311.pdb (deflated 78%)
  adding: content/output/EX_390.pdb (deflated 78%)
  adding: content/output/EX_360.pdb (deflated 78%)
  adding: content/output/EX_338.pdb (deflated 78%)
  adding: content/output/EX_315.pdb (deflated 78%)
  adding: content/output/EX_341.pdb (deflated 78%)
  adding: content/output/EX_363.pdb (deflated 78%)
  adding: content/output/EX_391.pdb (deflated 78%)
  adding: content/output/EX_378.pdb (deflated 78%)
  adding: content/output/EX_339.pdb (deflated 78%)
  adding: content/output/EX_379.pdb (deflated 78%)
  adding: content/output/EX_383.pdb (deflated 78%)
  adding: content/output/EX_397.pdb (deflated 78%)
  adding: content/output/EX_387.pdb (deflated 78%)
  adding: content/output/EX_370.pdb (deflated 78%)
  adding: content/output/EX_324.pdb (deflated 78%)
  adding: content/output/EX_305.pdb (deflated 78%)
  adding: content/output/EX_356.pdb (deflated 78%)
  adding: content/output/EX_343.pdb (deflate